# Decorator-Based Tracing for Azure OpenAI

## Introduction

This notebook demonstrates **non-intrusive tracing** using Python decorators. Instead of wrapping every API call manually, decorators allow you to add comprehensive tracing to existing functions without modifying their core logic.

### What are Decorators?

Python decorators are a powerful way to modify function behavior without changing the function's code. They wrap a function with additional functionality (like tracing) while keeping the original logic intact.

**Traditional Approach (Intrusive):**
```python
def my_function():
    with tracer.start_span("my_function"):  # Manual span management
        # Original logic here
        result = do_work()
        return result
```

**Decorator Approach (Non-Intrusive):**
```python
@trace_function(span_name="my_function")
def my_function():
    # Original logic here - no tracing code!
    result = do_work()
    return result
```

### Key Benefits

**Clean Code:**
- ✅ Business logic separated from observability concerns
- ✅ Functions remain readable and maintainable
- ✅ Easy to add/remove tracing without code changes

**Automatic Instrumentation:**
- ✅ Captures function arguments automatically
- ✅ Records return values and exceptions
- ✅ Measures execution time (latency)
- ✅ Handles both sync and async functions

**Composability:**
- ✅ Decorated functions can call other decorated functions
- ✅ Automatically creates parent-child span hierarchies
- ✅ Trace complete workflows end-to-end

**Flexibility:**
- ✅ Control what gets captured (args, results, messages)
- ✅ Custom attributes for business context
- ✅ Privacy controls for sensitive data

### Tracing Workflow

```
1. Setup Tracing (SQLite + OpenTelemetry)
   ↓
2. Decorate Functions (@trace_function, @trace_openai_call)
   ↓
3. Call Functions Normally (no code changes!)
   ↓
4. Traces Captured Automatically
   ↓
5. Query SQLite Database
   ↓
6. Analyze Performance & Debug Issues
```

### Available Decorators

**`@trace_openai_call`**: Specialized for Azure OpenAI API calls
- Captures model, temperature, max_tokens
- Records full prompts and completions (optional)
- Tracks token usage (prompt, completion, total)
- Measures API latency
- Handles streaming responses
- Traces function calling (tools, arguments, results)

**`@trace_function`**: Generic decorator for any function
- Captures function arguments
- Records return values
- Measures execution time
- Handles exceptions
- Works with sync and async functions
- Custom attributes support

### What Gets Traced

**Function Metadata:**
- Function name and module
- Execution duration (milliseconds)
- Success/failure status
- Exception details (if errors)

**Function Arguments:**
- All positional arguments
- All keyword arguments
- Serialized as strings (JSON for complex types)

**Return Values:**
- Complete return data
- Serialized for storage

**OpenAI-Specific (with `@trace_openai_call`):**
- Model/deployment name
- Request parameters (temperature, max_tokens, etc.)
- Full message history (system, user, assistant)
- Token usage breakdown
- Tool/function definitions
- Function call arguments and results

**Hierarchical Context:**
- Parent-child relationships
- Trace IDs (group related spans)
- Span IDs (unique per operation)
- Timestamps (start, end)

### Use Cases

**Development & Testing:**
- Debug function call sequences
- Validate function arguments
- Test error handling paths
- Profile performance locally

**Performance Analysis:**
- Identify slow functions
- Find bottlenecks in pipelines
- Compare different implementations
- Optimize critical paths

**Agent Workflows:**
- Trace complete agent orchestration
- Debug function calling behavior
- Validate tool argument passing
- Analyze multi-step reasoning

**Cost Tracking:**
- Monitor token usage per function
- Calculate API costs
- Identify expensive operations
- Optimize prompt efficiency

### Prerequisites

- Python 3.8+
- Azure OpenAI endpoint and API key
- OpenTelemetry packages (installed automatically)
- SQLite (built into Python)
- Decorator utilities (`tracing_decorators.py`)

## Table of Contents

1. [Part 1: Environment Setup](#part-1-environment-setup)
   - 1.1: Install Dependencies
   - 1.2: Configure Environment & Imports
   - 1.3: Initialize Tracing
   - 1.4: Initialize Azure OpenAI Clients
2. [Part 2: Basic Function Decoration](#part-2-basic-function-decoration)
3. [Part 3: Async Function Tracing](#part-3-async-function-tracing)
4. [Part 4: Generic Function Tracing](#part-4-generic-function-tracing)
5. [Part 5: Composing Traced Functions](#part-5-composing-traced-functions)
6. [Part 6: Privacy Controls](#part-6-privacy-controls)
7. [Part 7: Function Calling & Agent Orchestration](#part-7-function-calling--agent-orchestration)
8. [Part 8: Query and Analyze Traces](#part-8-query-and-analyze-traces)
9. [Summary and Best Practices](#summary-and-best-practices)

---

## Part 1: Environment Setup

### 1.1: Install Dependencies

Install required packages for decorator-based tracing.

In [ ]:
%pip install -qU openai python-dotenv  opentelemetry-sdk opentelemetry-api

### 1.2: Configure Environment & Imports

Load environment variables and import decorator utilities.

**Required Environment Variables:**
- `AZURE_OPENAI_ENDPOINT_GPT_4o`: Your Azure OpenAI endpoint
- `AZURE_OPENAI_API_KEY_GPT_4o`: Your Azure OpenAI API key

**Decorator Utilities:**
- `setup_tracing()`: Initialize OpenTelemetry with SQLite
- `trace_function()`: Generic function decorator
- `trace_openai_call()`: OpenAI-specific decorator
- `flush_traces()`: Force export pending spans

In [ ]:
import os
import logging
from dotenv import load_dotenv
from openai import AzureOpenAI, AsyncAzureOpenAI

# Suppress verbose logging
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("opentelemetry").setLevel(logging.WARNING)

# Import tracing decorators
from tracing_decorators import (
    setup_tracing,
    trace_function,
    trace_openai_call,
    flush_traces
)

# Load environment variables
load_dotenv("../.env")

# Verify environment variables
required_vars = ["AZURE_OPENAI_ENDPOINT_GPT_4o", "AZURE_OPENAI_API_KEY_GPT_4o"]
missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    print(f"⚠️  Warning: Missing environment variables: {missing}")
else:
    print("✅ Environment variables loaded successfully")
    print("✅ Decorator utilities imported successfully")

### 1.3: Initialize Tracing

Configure tracing with SQLite storage and service identification.

**Service Name Benefits:**
- Multiple applications can share the same database
- Filter traces by service for analysis
- Essential for microservices architectures
- Stored in both database column and span attributes

**Configuration:**
- `service_name`: Identifies this application
- `enable_console`: Print traces to console (default: False)
- `enable_sqlite`: Store in SQLite database (default: True)
- `sqlite_db_path`: Database file path
- `use_batch_processor`: Batch spans for performance (default: True)

In [ ]:
# Setup tracing once
tracer = setup_tracing(
    service_name="decorator-tracing-demo",
    enable_console=False,
    enable_sqlite=True,
    sqlite_db_path="decorator_traces.db",
    use_batch_processor=True,
    verbose=True
)

print("✅ Tracing initialized successfully")
print("📁 Traces will be stored in: decorator_traces.db")
print(f"🏢 Service name: decorator-tracing-demo")

### 1.4: Initialize Azure OpenAI Clients

Create synchronous and asynchronous clients for API calls.

In [ ]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT_GPT_4o"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY_GPT_4o"),
    api_version="2024-02-01"
)

async_client = AsyncAzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT_GPT_4o"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY_GPT_4o"),
    api_version="2024-02-01"
)

deployment_name = "gpt-4o-lei"

print("✅ Azure OpenAI clients initialized")
print(f"   Deployment: {deployment_name}")
print(f"   Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT_GPT_4o')}")

---

## Part 2: Basic Function Decoration

Add tracing to existing Azure OpenAI functions without modifying their core logic.

### The Decorator Pattern

**Before (No Tracing):**
```python
def ask_gpt(client, messages, model):
    return client.chat.completions.create(
        model=model,
        messages=messages
    )
```

**After (With Tracing):**
```python
@trace_openai_call(operation="chat.completions")  # Just add this line!
def ask_gpt(client, messages, model):
    return client.chat.completions.create(
        model=model,
        messages=messages
    )
```

The function logic remains **identical** - tracing is added automatically!

### What Gets Captured

- ✅ Function name and arguments
- ✅ Model and request parameters
- ✅ Full message history
- ✅ Token usage (prompt, completion, total)
- ✅ Response content
- ✅ Execution latency
- ✅ Errors and exceptions

In [ ]:
# Your original function - just add the decorator!
@trace_openai_call(operation="chat.completions")
def ask_gpt(client, messages, model, temperature=0.7, max_tokens=200):
    """Simple chat completion function"""
    return client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )


# Use it normally - tracing happens automatically!
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is Azure OpenAI?"}
]

response = ask_gpt(
    client=client,
    messages=messages,
    model=deployment_name,
    temperature=0.7,
    max_tokens=200
)

print(f"📊 Performance Metrics:")
print(f"   Tokens: {response.usage.total_tokens} (prompt: {response.usage.prompt_tokens}, completion: {response.usage.completion_tokens})")
print(f"   Finish reason: {response.choices[0].finish_reason}")

print(f"\n💬 Response:\n{response.choices[0].message.content}")

print("\n✅ Trace automatically captured and stored in database")
print("   No manual span management required!")

---

## Part 3: Async Function Tracing

The `@trace_openai_call` decorator automatically detects and handles async functions.

### Automatic Async Detection

The decorator uses `inspect.iscoroutinefunction()` to determine if a function is async, then:
- Returns an async wrapper for async functions
- Returns a sync wrapper for sync functions

**No configuration needed** - it just works!

### Benefits of Async Tracing

- ✅ Non-blocking I/O for better concurrency
- ✅ Higher throughput for multiple API calls
- ✅ Same tracing features as sync version
- ✅ Identical decorator syntax

In [ ]:
@trace_openai_call(operation="chat.completions")
async def ask_gpt_async(client, messages, model, temperature=0.7, max_tokens=200):
    """Async chat completion function"""
    return await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )


# Use it normally with await
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Explain distributed tracing briefly."}
]

response = await ask_gpt_async(
    client=async_client,
    messages=messages,
    model=deployment_name,
    temperature=0.7,
    max_tokens=250
)

print(f"📊 Performance Metrics:")
print(f"   Tokens: {response.usage.total_tokens}")
print(f"   Model: {deployment_name}")

print(f"\n💬 Response:\n{response.choices[0].message.content}")

print("\n✅ Async trace automatically captured")
print("   Decorator detected async function and handled it correctly!")

---

## Part 4: Generic Function Tracing

Use `@trace_function` for any Python function, not just OpenAI API calls.

### Universal Function Decorator

The `@trace_function` decorator works with:
- Data processing functions
- Business logic
- Database operations
- File I/O operations
- Mathematical computations
- Any Python function!

### Custom Attributes

Add business context to spans with custom attributes:
```python
@trace_function(span_name="process_order", attributes={"version": "2.0", "environment": "prod"})
def process_order(order_id):
    # Your logic here
    pass
```

### What Gets Captured

- ✅ Function arguments (all types)
- ✅ Return values
- ✅ Execution duration
- ✅ Exceptions with stack traces
- ✅ Custom attributes

In [ ]:
@trace_function(span_name="data_processing", attributes={"version": "1.0"})
def process_data(text: str, convert_upper: bool = True):
    """Example data processing function"""
    import time
    time.sleep(0.1)  # Simulate work
    
    if convert_upper:
        return text.upper()
    return text.lower()


@trace_function(span_name="text_analysis")
def analyze_text(text: str):
    """Analyze text properties"""
    return {
        "length": len(text),
        "words": len(text.split()),
        "has_numbers": any(c.isdigit() for c in text)
    }


# Use them normally - tracing is automatic
result1 = process_data("hello world", convert_upper=True)
print(f"📝 Processed: {result1}")

result2 = analyze_text("Azure OpenAI has 3 main features")
print(f"📊 Analysis: {result2}")

print("\n✅ Both function calls traced automatically")
print("   Check the database to see:")
print("   - Function arguments (text, convert_upper)")
print("   - Return values (results)")
print("   - Execution duration")
print("   - Custom attributes (version: 1.0)")

---

## Part 5: Composing Traced Functions

Create pipelines of traced functions that automatically form parent-child span hierarchies.

### Span Hierarchy

When decorated functions call other decorated functions, OpenTelemetry automatically:
- Creates parent-child relationships
- Links spans with the same trace_id
- Tracks the complete call stack
- Preserves context across async boundaries

**Example Hierarchy:**
```
complete_pipeline (root span)
  ├── validate_input (child span)
  ├── prepare_messages (child span)
  └── call_openai (child span)
```

### Benefits

- ✅ **Complete visibility**: See entire workflow in one trace
- ✅ **Debugging**: Identify which step failed
- ✅ **Performance**: Find bottlenecks in the pipeline
- ✅ **Context**: Understand the call sequence

In [ ]:
@trace_function(span_name="validate_input")
def validate_input(text: str) -> bool:
    """Validate input text"""
    return len(text) > 0 and len(text) < 1000


@trace_function(span_name="prepare_messages")
def prepare_messages(user_input: str, system_prompt: str = None):
    """Prepare messages for OpenAI"""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_input})
    return messages


@trace_openai_call(operation="chat.completions")
def call_openai(client, messages, model, temperature=0.7):
    """Call OpenAI API"""
    return client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=150
    )


@trace_function(span_name="complete_pipeline")
def process_user_query(user_input: str, client, model: str):
    """Complete pipeline with validation, preparation, and API call"""

    # Each of these calls creates its own span
    if not validate_input(user_input):
        raise ValueError("Invalid input")

    messages = prepare_messages(
        user_input=user_input,
        system_prompt="You are a concise AI assistant."
    )

    response = call_openai(
        client=client,
        messages=messages,
        model=model,
        temperature=0.7
    )

    return response.choices[0].message.content


# Run the pipeline - you'll see a hierarchy of spans!
result = process_user_query(
    user_input="What are the benefits of observability?",
    client=client,
    model=deployment_name
)

print(f"💬 Final Result:\n{result}")

print("\n✅ Complete pipeline traced with span hierarchy:")
print("   📊 complete_pipeline (root)")
print("      ├─ validate_input")
print("      ├─ prepare_messages")
print("      └─ call_openai")
print("\n   Query the database to see parent-child relationships!")

---

## Part 6: Privacy Controls

Control what data gets captured in traces for compliance and privacy.

### Capture Control Options

**`@trace_openai_call` Parameters:**
- `capture_messages`: Capture full prompt messages (default: True)
- `capture_response`: Capture completion content (default: True)
- `operation`: Operation name for span

**Use Cases:**
- **Development**: Enable all capture for debugging
- **Production**: Disable sensitive data capture
- **Compliance**: Meet GDPR/HIPAA requirements
- **Cost Optimization**: Reduce storage with selective capture

### Example Configurations

```python
# Full capture (development)
@trace_openai_call(operation="chat", capture_messages=True, capture_response=True)

# No message capture (protect PII)
@trace_openai_call(operation="chat", capture_messages=False, capture_response=True)

# Metadata only (production)
@trace_openai_call(operation="chat", capture_messages=False, capture_response=False)
```

**Still captured when disabled:**
- Token counts
- Latency metrics
- Model parameters
- Success/failure status

In [ ]:
# Don't capture messages (for privacy)
@trace_openai_call(operation="chat.completions", capture_messages=False)
def private_chat(client, messages, model):
    return client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=100
    )


# Don't capture response content
@trace_openai_call(operation="chat.completions", capture_response=False)
def chat_no_response_capture(client, messages, model):
    return client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=100
    )


# Test both configurations
messages = [
    {"role": "user", "content": "What is Python?"}
]

response1 = private_chat(client, messages, deployment_name)
print(f"🔒 Response 1 (messages NOT captured for privacy):")
print(f"   {response1.choices[0].message.content[:50]}...")

response2 = chat_no_response_capture(client, messages, deployment_name)
print(f"\n🔒 Response 2 (response NOT captured for privacy):")
print(f"   {response2.choices[0].message.content[:50]}...")

print("\n✅ Privacy controls applied:")
print("   - Trace 1: No message content stored (protect user input)")
print("   - Trace 2: No response content stored (protect AI output)")
print("   - Both still capture: tokens, latency, model, status")

---

## Part 7: Function Calling & Agent Orchestration

Trace complete agent workflows including function calling with tools.

### Function Calling Flow

```
1. User Query → Agent
   ↓
2. Agent Requests Tool Call(s)
   ↓
3. Execute Local Functions
   ↓
4. Return Results to Agent
   ↓
5. Agent Generates Final Response
```

Each step is automatically traced with the decorator pattern!

### What Gets Traced

**Tool Definitions:**
- Function names and descriptions
- Parameter schemas
- Required vs optional parameters

**Function Execution:**
- Function name and arguments
- Return values
- Execution duration
- Success/failure status

**Agent Interactions:**
- Initial request with tools
- Tool call requests from agent
- Function results submission
- Final response generation

### Hierarchical Trace

The complete workflow creates a trace hierarchy:
```
gpt_function_calling_orchestration (root)
  ├─ chat.completions.with_tools
  ├─ execute_function_call (get_weather)
  ├─ execute_function_call (get_stock_price)
  └─ chat.completions.with_function_result
```

In [ ]:
import json

# Define tools/functions
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get the current stock price for a company",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The stock ticker symbol, e.g. MSFT for Microsoft"
                    }
                },
                "required": ["symbol"]
            }
        }
    }
]

# Simulated function implementations


def get_weather(location: str, unit: str = "fahrenheit") -> str:
    """Simulate getting weather data"""
    return json.dumps({
        "location": location,
        "temperature": 72 if unit == "fahrenheit" else 22,
        "unit": unit,
        "condition": "sunny"
    })


def get_stock_price(symbol: str) -> str:
    """Simulate getting stock price"""
    prices = {"MSFT": 380.50, "AAPL": 195.30, "GOOGL": 142.80}
    return json.dumps({
        "symbol": symbol,
        "price": prices.get(symbol.upper(), 100.00),
        "currency": "USD"
    })


# Map function names to implementations
available_functions = {
    "get_weather": get_weather,
    "get_stock_price": get_stock_price
}

print("✅ Functions defined")

In [ ]:
# Decorated function for initial GPT call with tools
@trace_openai_call(operation="chat.completions.with_tools")
def call_gpt_with_tools(client, messages, model, tools):
    """Call GPT with function calling enabled"""
    return client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )


# Decorated function for executing local functions
@trace_function(span_name="execute_function_call")
def execute_function_call(function_name: str, function_args: dict):
    """Execute a function call locally"""
    if function_name in available_functions:
        function_to_call = available_functions[function_name]
        return function_to_call(**function_args)
    else:
        return json.dumps({"error": f"Function {function_name} not found"})


# Decorated function for sending function result back to GPT
@trace_openai_call(operation="chat.completions.with_function_result")
def call_gpt_with_function_result(client, messages, model):
    """Send function result back to GPT"""
    return client.chat.completions.create(
        model=model,
        messages=messages
    )


# Complete orchestration with tracing
@trace_function(span_name="gpt_function_calling_orchestration")
def run_agent_with_function_calling(user_query: str):
    """
    Complete agent orchestration with function calling
    This creates a hierarchy of spans showing the entire flow
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant with access to weather and stock price data."},
        {"role": "user", "content": user_query}
    ]

    print(f"\n🤖 User Query: {user_query}\n")

    # Step 1: Call GPT with tools
    response = call_gpt_with_tools(
        client=client,
        messages=messages,
        model=deployment_name,
        tools=tools
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # If no tool calls, return the response directly
    if not tool_calls:
        print(f"✅ Direct Response: {response_message.content}")
        return response_message.content

    # Add assistant's response to messages
    messages.append(response_message)

    # Step 2: Execute each function call
    print(f"🔧 Function Calls Detected: {len(tool_calls)}\n")

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        print(f"📞 Calling: {function_name}({function_args})")

        # Execute the function (traced!)
        function_response = execute_function_call(function_name, function_args)

        print(f"📊 Result: {function_response}\n")

        # Add function result to messages
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response
        })

    # Step 3: Send function results back to GPT
    print("🔄 Sending results back to GPT...\n")

    final_response = call_gpt_with_function_result(
        client=client,
        messages=messages,
        model=deployment_name
    )

    final_content = final_response.choices[0].message.content
    print(f"✅ Final Response: {final_content}\n")

    return final_content


# Test the agent!
result = run_agent_with_function_calling(
    "What's the weather in San Francisco and what's the current price of Microsoft stock?"
)

print("\n" + "="*80)
print(f"💬 Final Result:\n{result}")
print("="*80)

### What Gets Traced?

The function calling example creates a **hierarchical trace** showing:

1. **`gpt_function_calling_orchestration`** - Root span for entire flow
   - **`chat.completions.with_tools`** - Initial GPT call with tool definitions
   - **`execute_function_call`** - Each function execution (get_weather, get_stock_price)
   - **`chat.completions.with_function_result`** - Final GPT call with function results

Each span captures:
- ⏱️ **Latency** - How long each step took
- 🔗 **Parent-child relationships** - See the complete call hierarchy
- 📊 **Tokens** - Token usage for GPT calls
- 🛠️ **Function details** - Which functions were called with what arguments
- ✅ **Success/failure** - Status of each operation

This gives you complete visibility into your agent's behavior!

### 🔍 Viewing Function Arguments in Database

After running the function calling example, you can query the database to see the actual function arguments:

```sql
SELECT key, value 
FROM span_attributes 
WHERE span_id IN (
    SELECT id FROM spans WHERE name = 'execute_function_call'
)
AND key LIKE 'arg.%';
```

You'll see entries like:
- `arg.function_name`: `"get_weather"`
- `arg.function_args`: `{"location": "San Francisco", "unit": "fahrenheit"}` (as JSON string)

The decorator now serializes dictionaries and lists as JSON strings, so you can see the complete argument values!

In [ ]:
import sqlite3
import pandas as pd


def view_function_call_details(db_path="decorator_traces.db"):
    """View detailed function call arguments from traces"""
    conn = sqlite3.connect(db_path)

    print("🔍 Function Call Details")
    print("=" * 80)

    # Query function call spans with their arguments
    query = """
        SELECT 
            s.id,
            s.name,
            s.created_at,
            sa.key,
            sa.value
        FROM spans s
        LEFT JOIN span_attributes sa ON s.id = sa.span_id
        WHERE s.name = 'execute_function_call'
        AND sa.key LIKE 'arg.%'
        ORDER BY s.created_at DESC, sa.key
    """

    df = pd.read_sql_query(query, conn)

    if len(df) > 0:
        # Group by span to show each function call
        for span_id in df['id'].unique():
            span_data = df[df['id'] == span_id]
            print(f"\n📞 Function Call (Span ID: {span_id})")
            print(f"   Timestamp: {span_data.iloc[0]['created_at']}")
            print(f"   Arguments:")

            for _, row in span_data.iterrows():
                key = row['key'].replace('arg.', '')
                value = row['value']

                # Try to pretty print JSON
                if value and (value.startswith('{') or value.startswith('[')):
                    try:
                        parsed = json.loads(value)
                        value = json.dumps(parsed, indent=6)
                        print(f"     {key}:")
                        for line in value.split('\n'):
                            print(f"       {line}")
                    except:
                        print(f"     {key}: {value}")
                else:
                    print(f"     {key}: {value}")

        print("\n" + "=" * 80)
    else:
        print("No function call traces found.")
        print("Make sure to run Example 6 first and flush traces!")
        print("=" * 80)

    conn.close()


# View the function call details
view_function_call_details()

### 🔗 Grouping Spans by Trace

Each execution of `run_agent_with_function_calling` creates a **trace** - a collection of related spans with the same `trace_id`. This lets you see all the spans that belong to a single function call execution.

In [ ]:
def view_traces_grouped(db_path="decorator_traces.db", limit=3):
    """View traces grouped by trace_id showing complete execution flows"""
    conn = sqlite3.connect(db_path)
    
    print("🔗 Traces Grouped by Execution")
    print("=" * 80)
    
    # Get unique traces for the orchestration function
    traces_query = """
        SELECT DISTINCT trace_id, MIN(created_at) as first_span_time
        FROM spans 
        WHERE name = 'gpt_function_calling_orchestration'
        ORDER BY first_span_time DESC
        LIMIT ?
    """
    
    cursor = conn.cursor()
    cursor.execute(traces_query, (limit,))
    traces = cursor.fetchall()
    
    if not traces:
        print("No function calling traces found.")
        print("Make sure to run Example 6 first!")
        print("=" * 80)
        conn.close()
        return
    
    for idx, (trace_id, first_time) in enumerate(traces, 1):
        print(f"\n📊 Trace #{idx}: {trace_id}")
        print(f"   Started: {first_time}")
        print(f"   {'─' * 76}")
        
        # Get all spans for this trace
        spans_query = """
            SELECT 
                id,
                name,
                ROUND(duration_ms, 2) as duration_ms,
                parent_span_id,
                status_code,
                datetime(created_at) as created_at
            FROM spans 
            WHERE trace_id = ?
            ORDER BY created_at ASC
        """
        
        df = pd.read_sql_query(spans_query, conn, params=(trace_id,))
        
        print(f"\n   Total spans in this trace: {len(df)}")
        print(f"\n   Span Hierarchy:")
        
        # Build hierarchy visualization
        for _, row in df.iterrows():
            indent = "     "
            if row['parent_span_id'] is None:
                # Root span
                print(f"   {indent}┌─ {row['name']}")
            else:
                # Child span
                print(f"   {indent}├─── {row['name']}")
            
            print(f"   {indent}│    Duration: {row['duration_ms']}ms | Status: {row['status_code']}")
        
        # Get total duration
        total_duration = df['duration_ms'].iloc[0] if len(df) > 0 else 0
        print(f"\n   ⏱️  Total Duration: {total_duration}ms")
        
        # Count function calls
        function_calls = len(df[df['name'] == 'execute_function_call'])
        openai_calls = len(df[df['name'].str.contains('chat.completions', na=False)])
        
        print(f"   📞 Function Calls: {function_calls}")
        print(f"   🤖 OpenAI API Calls: {openai_calls}")
        print(f"   {'─' * 76}")
    
    conn.close()
    print("\n" + "=" * 80)

# View the most recent traces
view_traces_grouped(limit=3)

In [ ]:
def view_trace_details(trace_id: str = None, db_path="decorator_traces.db"):
    """View detailed hierarchy for a specific trace"""
    conn = sqlite3.connect(db_path)
    
    # If no trace_id provided, get the most recent one
    if trace_id is None:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT trace_id 
            FROM spans 
            WHERE name = 'gpt_function_calling_orchestration'
            ORDER BY created_at DESC 
            LIMIT 1
        """)
        result = cursor.fetchone()
        if not result:
            print("No traces found!")
            conn.close()
            return
        trace_id = result[0]
    
    print("🔍 Detailed Trace View")
    print("=" * 80)
    print(f"Trace ID: {trace_id}\n")
    
    # Get all spans for this trace
    query = """
        SELECT 
            id,
            span_id,
            parent_span_id,
            name,
            ROUND(duration_ms, 2) as duration_ms,
            status_code,
            datetime(created_at) as created_at
        FROM spans 
        WHERE trace_id = ?
        ORDER BY created_at ASC
    """
    
    df = pd.read_sql_query(query, conn, params=(trace_id,))
    
    # Build parent-child map
    span_map = {}
    for _, row in df.iterrows():
        span_map[row['span_id']] = {
            'id': row['id'],
            'name': row['name'],
            'duration': row['duration_ms'],
            'status': row['status_code'],
            'parent': row['parent_span_id'],
            'children': []
        }
    
    # Find root and build tree
    root = None
    for span_id, span_data in span_map.items():
        parent_id = span_data['parent']
        if parent_id is None:
            root = span_id
        elif parent_id in span_map:
            span_map[parent_id]['children'].append(span_id)
    
    # Recursive function to print tree
    def print_span_tree(span_id, indent=0):
        span = span_map[span_id]
        prefix = "  " * indent
        
        if indent == 0:
            symbol = "📊"
        elif len(span['children']) > 0:
            symbol = "├─"
        else:
            symbol = "└─"
        
        print(f"{prefix}{symbol} {span['name']}")
        print(f"{prefix}   ⏱️  {span['duration']}ms | ✅ {span['status']} | ID: {span['id']}")
        
        for child_id in span['children']:
            print_span_tree(child_id, indent + 1)
    
    if root:
        print_span_tree(root)
    else:
        print("Could not build hierarchy - no root span found")
    
    print("\n" + "=" * 80)
    print(f"Summary: {len(df)} spans in this trace")
    print("=" * 80)
    
    conn.close()

# View the most recent trace in detail
view_trace_details()

### 💡 Understanding Traces vs Spans

**Key Concept:**
- **Trace** = One complete execution flow (identified by `trace_id`)
- **Span** = One operation within that flow (function call, API call, etc.)

**Example:** When you run `run_agent_with_function_calling()` once:
- ✅ Creates **1 trace** with a unique `trace_id`
- ✅ That trace contains **5 spans**:
  1. `gpt_function_calling_orchestration` (root span)
  2. `chat.completions.with_tools` (child)
  3. `execute_function_call` for get_weather (child)
  4. `execute_function_call` for get_stock_price (child)
  5. `chat.completions.with_function_result` (child)

All 5 spans share the **same `trace_id`**, so you can query them together!

**In your database:**
```sql
-- Find all spans for a specific trace
SELECT * FROM spans WHERE trace_id = 'abc123...'

-- Count spans per trace
SELECT trace_id, COUNT(*) as span_count 
FROM spans 
GROUP BY trace_id
```

---

## Part 8: Query and Analyze Traces

Flush pending traces and query the SQLite database for analysis.

### Flush Traces

Force export of all pending spans (important with BatchSpanProcessor).

In [ ]:
# Force flush all pending traces
flush_traces(timeout=10)
print("✅ All pending traces flushed to database")
print("📊 Ready for querying and analysis")

### Query Trace Summary

View all captured traces with service breakdown, hierarchy, and performance metrics.

In [ ]:
import sqlite3
import pandas as pd

def query_traces(db_path="decorator_traces.db"):
    """Query trace database"""
    conn = sqlite3.connect(db_path)
    
    print("=" * 80)
    print("📊 TRACE SUMMARY")
    print("=" * 80)
    
    # Get total count
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM spans")
    total_spans = cursor.fetchone()[0]
    print(f"\n📈 Total spans: {total_spans}")
    
    # Check if service_name column exists (for backward compatibility)
    cursor.execute("PRAGMA table_info(spans)")
    columns = [col[1] for col in cursor.fetchall()]
    has_service_name = 'service_name' in columns
    
    # Show service breakdown if service_name exists
    if has_service_name:
        print("\n🏢 Traces by Service:")
        df_services = pd.read_sql_query("""
            SELECT 
                service_name,
                COUNT(*) as count,
                ROUND(AVG(duration_ms), 2) as avg_duration_ms
            FROM spans 
            GROUP BY service_name
        """, conn)
        print(df_services.to_string(index=False))
    
    # Get latest spans with hierarchy
    print("\n🕐 Latest Traces (showing parent-child relationships):")
    service_col = "service_name," if has_service_name else ""
    df = pd.read_sql_query(f"""
        SELECT 
            id,
            {service_col}
            name,
            ROUND(duration_ms, 2) as duration_ms,
            parent_span_id,
            datetime(created_at) as created_at
        FROM spans 
        ORDER BY created_at DESC 
        LIMIT 15
    """, conn)
    print(df.to_string(index=False))
    
    # Performance by operation
    print("\n⚡ Performance by Operation:")
    df_perf = pd.read_sql_query("""
        SELECT 
            name,
            COUNT(*) as count,
            ROUND(AVG(duration_ms), 2) as avg_ms,
            ROUND(MIN(duration_ms), 2) as min_ms,
            ROUND(MAX(duration_ms), 2) as max_ms
        FROM spans 
        GROUP BY name
        ORDER BY avg_ms DESC
    """, conn)
    print(df_perf.to_string(index=False))
    
    conn.close()
    print("\n" + "=" * 80)

query_traces()

## Filter Traces by Service Name

Now you can see the `service_name` column in the database! This is useful when you have multiple services writing to the same database.

### 📚 Best Practice: Hybrid Storage

The implementation follows **industry best practices** by storing `service_name` in two places:

1. **Database Column** (`spans.service_name`)
   - ✅ Fast filtering and querying
   - ✅ Indexed for performance
   - ✅ Used by Jaeger, Zipkin, Tempo, SignalFx
   
2. **Span Attribute** (`service.name`)
   - ✅ OpenTelemetry standard compliance
   - ✅ Compatible with OTel collectors
   - ✅ Follows semantic conventions

This hybrid approach gives you the best of both worlds: **fast queries** + **OTel compatibility**.

In [ ]:
def query_by_service(service_name: str, db_path="decorator_traces.db"):
    """Query traces for a specific service"""
    conn = sqlite3.connect(db_path)
    
    print(f"🔍 Traces for service: {service_name}")
    print("=" * 80)
    
    df = pd.read_sql_query("""
        SELECT 
            id,
            name,
            ROUND(duration_ms, 2) as duration_ms,
            status_code,
            datetime(created_at) as created_at
        FROM spans 
        WHERE service_name = ?
        ORDER BY created_at DESC
        LIMIT 10
    """, conn, params=(service_name,))
    
    if len(df) > 0:
        print(df.to_string(index=False))
    else:
        print(f"No traces found for service: {service_name}")
    
    conn.close()
    print("=" * 80)

# Query for our specific service
query_by_service("decorator-tracing-demo")

### Query Comparison: Column vs Attribute

Let's compare query performance between the two approaches:

In [ ]:
import time

def compare_query_methods(service_name: str, db_path="decorator_traces.db"):
    """Compare query performance: column vs attribute"""
    conn = sqlite3.connect(db_path)
    
    print("🏁 Query Performance Comparison")
    print("=" * 80)
    
    # Method 1: Query by column (FAST)
    start = time.time()
    df1 = pd.read_sql_query("""
        SELECT COUNT(*) as count
        FROM spans 
        WHERE service_name = ?
    """, conn, params=(service_name,))
    time1 = (time.time() - start) * 1000
    
    print(f"\n✅ Method 1: Column-based query")
    print(f"   SELECT * FROM spans WHERE service_name = '{service_name}'")
    print(f"   Time: {time1:.3f}ms")
    print(f"   Result: {df1['count'].values[0]} spans")
    
    # Method 2: Query by attribute (SLOWER - requires JOIN)
    start = time.time()
    df2 = pd.read_sql_query("""
        SELECT COUNT(*) as count
        FROM spans s
        INNER JOIN span_attributes sa ON s.id = sa.span_id
        WHERE sa.key = 'service.name' AND sa.value = ?
    """, conn, params=(service_name,))
    time2 = (time.time() - start) * 1000
    
    print(f"\n📊 Method 2: Attribute-based query (OTel compatible)")
    print(f"   SELECT * FROM spans JOIN span_attributes WHERE key='service.name'")
    print(f"   Time: {time2:.3f}ms")
    print(f"   Result: {df2['count'].values[0]} spans")
    
    # Show speedup
    speedup = time2 / time1 if time1 > 0 else 0
    print(f"\n⚡ Speedup: {speedup:.1f}x faster with column-based query")
    print(f"   💡 This is why we store service_name as a column!")
    
    conn.close()
    print("=" * 80)

# Run comparison
compare_query_methods("decorator-tracing-demo")

---

## Summary and Best Practices

### Key Takeaways

1. **Non-Intrusive Tracing**: Add observability without modifying function logic
2. **Clean Code**: Separation of concerns (business logic vs. tracing)
3. **Automatic Detection**: Handles sync and async functions automatically
4. **Composability**: Creates span hierarchies for complex workflows
5. **Privacy Controls**: Configurable capture for compliance
6. **Universal Application**: Works with any Python function

### Decorator Comparison

| Feature | @trace_openai_call | @trace_function |
|---------|-------------------|-----------------|
| **Purpose** | Azure OpenAI API calls | Any Python function |
| **Captures Model** | ✅ Yes | ❌ No |
| **Captures Tokens** | ✅ Yes | ❌ No |
| **Captures Messages** | ✅ Optional | ❌ No |
| **Captures Response** | ✅ Optional | ❌ No |
| **Captures Arguments** | ✅ Yes | ✅ Yes |
| **Captures Return Value** | ✅ Yes | ✅ Yes |
| **Measures Latency** | ✅ Yes | ✅ Yes |
| **Handles Exceptions** | ✅ Yes | ✅ Yes |
| **Async Support** | ✅ Yes | ✅ Yes |
| **Custom Attributes** | ✅ Via operation | ✅ Via attributes param |

### When to Use Each Decorator

**Use `@trace_openai_call` for:**
- Azure OpenAI API calls
- Chat completions (with or without streaming)
- Function calling / tool usage
- When you need token usage tracking
- When you want message/response capture

**Use `@trace_function` for:**
- Data processing functions
- Business logic
- Validation functions
- Helper utilities
- Database operations
- File I/O
- Any non-OpenAI function

### Best Practices

#### 1. Decorator Placement
- ✅ Apply decorators to public-facing functions
- ✅ Trace complete workflows, not every helper
- ✅ Use custom span names for clarity
- ✅ Add business context with custom attributes
- ❌ Avoid over-instrumentation (too many spans)

#### 2. Privacy & Compliance
- ✅ Disable message capture in production (`capture_messages=False`)
- ✅ Disable response capture for sensitive data
- ✅ Review captured data for PII
- ✅ Implement data retention policies
- ✅ Use separate databases for dev/prod

#### 3. Performance
- ✅ Use `use_batch_processor=True` for better performance
- ✅ Call `flush_traces()` before script/notebook ends
- ✅ Monitor database size for long-running applications
- ✅ Consider sampling for very high-volume functions
- ❌ Don't flush after every single call

#### 4. Span Hierarchy
- ✅ Use descriptive span names
- ✅ Compose decorated functions for workflows
- ✅ Query by `trace_id` to see complete flows
- ✅ Use `parent_span_id` to understand call stacks
- ✅ Group related operations in a root span

#### 5. Service Identification
- ✅ Use meaningful service names
- ✅ Filter traces by service in shared databases
- ✅ Follow naming conventions (lowercase, hyphens)
- ✅ Include environment in service name if needed
- ✅ Use consistent naming across your organization

#### 6. Function Calling
- ✅ Decorate tool definition functions
- ✅ Trace function execution (arguments, results)
- ✅ Create orchestration root span
- ✅ Capture tool call metadata
- ✅ Link function results to tool calls

### Code Patterns

**Simple Function:**
```python
@trace_function(span_name="process_data")
def process_data(input: str) -> dict:
    # Your logic here
    return result
```

**OpenAI Call:**
```python
@trace_openai_call(operation="chat.completions")
def ask_gpt(client, messages, model):
    return client.chat.completions.create(
        model=model,
        messages=messages
    )
```

**Privacy-Aware:**
```python
@trace_openai_call(
    operation="chat.completions",
    capture_messages=False,  # Don't store prompts
    capture_response=False   # Don't store completions
)
def private_chat(client, messages, model):
    return client.chat.completions.create(...)
```

**With Custom Attributes:**
```python
@trace_function(
    span_name="process_order",
    attributes={
        "version": "2.0",
        "environment": "production",
        "region": "us-west-2"
    }
)
def process_order(order_id: str):
    # Your logic here
    pass
```

**Agent Orchestration:**
```python
@trace_function(span_name="agent_workflow")
def run_agent(user_query: str):
    # Each decorated call creates a child span
    validated = validate_input(user_query)  # @trace_function
    messages = prepare_messages(validated)  # @trace_function
    response = call_openai(messages)        # @trace_openai_call
    result = post_process(response)         # @trace_function
    return result
```

### Database Queries

**Find slow functions:**
```sql
SELECT name, duration_ms 
FROM spans 
WHERE duration_ms > 1000
ORDER BY duration_ms DESC;
```

**Trace hierarchy:**
```sql
SELECT id, name, parent_span_id, duration_ms
FROM spans
WHERE trace_id = 'your_trace_id'
ORDER BY id;
```

**Service breakdown:**
```sql
SELECT service_name, COUNT(*) as count, AVG(duration_ms) as avg_ms
FROM spans
GROUP BY service_name;
```

**Function call arguments:**
```sql
SELECT s.name, sa.key, sa.value
FROM spans s
JOIN span_attributes sa ON s.id = sa.span_id
WHERE sa.key LIKE 'arg.%'
ORDER BY s.created_at DESC;
```

### Advantages Over Manual Instrumentation

| Aspect | Manual Instrumentation | Decorator-Based |
|--------|----------------------|-----------------|
| **Code Changes** | Many (wrap every call) | Minimal (one line) |
| **Readability** | Lower (mixed concerns) | Higher (separated) |
| **Maintainability** | Harder (scattered code) | Easier (centralized) |
| **Consistency** | Error-prone | Guaranteed |
| **Async Handling** | Manual detection | Automatic |
| **Error Handling** | Must implement | Built-in |
| **Testing** | Complex (mock spans) | Simple (mock function) |

### Migration Strategy

**From No Tracing:**
1. Start with critical functions (`@trace_function`)
2. Add OpenAI decorators (`@trace_openai_call`)
3. Test with sample workloads
4. Review captured data
5. Roll out to production

**From Manual Tracing:**
1. Identify manually instrumented functions
2. Remove manual span creation code
3. Add appropriate decorator
4. Test that spans are still created
5. Clean up imports and helper code

**To Cloud Tracing:**
1. Keep decorators (they work with any exporter)
2. Replace `setup_tracing()` with Azure Monitor config
3. Update connection string
4. Test export to Application Insights
5. Maintain local tracing for development

### Troubleshooting

| Issue | Solution |
|-------|----------|
| Decorator not tracing | Check `setup_tracing()` was called first |
| Missing spans | Call `flush_traces()` before querying |
| No async traces | Ensure async function uses `await` |
| Function args truncated | Check decorator serialization logic |
| Performance impact | Use `use_batch_processor=True` |
| Database too large | Implement data retention policy |

### Next Steps

1. **Experiment**: Add decorators to your own functions
2. **Query**: Explore the database with SQL/Pandas
3. **Analyze**: Find slow functions and bottlenecks
4. **Optimize**: Use insights to improve performance
5. **Integrate**: Connect to your CI/CD pipeline
6. **Scale**: Migrate to cloud tracing for production

### Additional Resources

- [OpenTelemetry Python SDK](https://opentelemetry.io/docs/instrumentation/python/)
- [Python Decorators Guide](https://realpython.com/primer-on-python-decorators/)
- [Azure Monitor OpenTelemetry](https://learn.microsoft.com/azure/azure-monitor/app/opentelemetry-enable)
- [Semantic Conventions](https://opentelemetry.io/docs/specs/semconv/)

### Related Notebooks

- `02_local_tracing.ipynb`: Manual tracing with wrapper functions
- `01_azure_foundry_tracing.ipynb`: Cloud-based tracing with Application Insights
- `../01_agent/`: Azure AI agent examples
- `../05_evaluation/`: Agent evaluation and metrics